# Preliminaries

In [ ]:
# Install Reka API.
!python3 -m pip install --quiet reka-api

In [ ]:
from IPython.display import Image
import os
import reka
import time

from getpass import getpass

# Assuming this is the IP of a VM with on-prem deployment
# reka._SERVER = "http://***********:8000"
# reka.API_KEY = "on_prem_key"

# To use the prod server:
reka._SERVER = "https://api.reka.ai"
reka.API_KEY = getpass("Enter your Reka API key: ")

# Chat capabilities

In [ ]:
# Very simple chat interface.
response = reka.chat("What is the capital of the UK?")
print(response['text'])

In [ ]:
# You can also specify advanced parameters to control generation.
response = reka.chat(
    "List some large cities in the UK.",
    assistant_start_text="1.",
    request_output_len=128,
    temperature=0.4,
    stop_tokens=["11."],
)

# Retrieval capabilities

In [ ]:
# Download a toy dataset.
!wget https://reka-public.s3.eu-west-2.amazonaws.com/amazon_legal_docs.zip

In [ ]:
reka.add_dataset(
    filepath="amazon_legal_docs.zip",
    name="amazon",
)

In [ ]:
# Prepare the documents for retrieval.
job_id = reka.prepare_retrieval(dataset_name="amazon")
while True:
    status = reka.retrieval_job_status(job_id)
    if status.is_done():
        break
    time.sleep(5)
print(f"Job finished with status: {status.job_status.name}")

In [ ]:
# The model knows everything about Amazon legal documentation.
response = reka.chat(
    'What is the refund policy for Amazon Prime memberships that were redeemed through a promotional or gift code?',
    retrieval_dataset="amazon")
print(response["text"])

In [ ]:
# Cleanup
reka.delete_dataset("amazon")

# Image understanding

In [ ]:
# Let's use a random image
URL = "https://docs.reka.ai/_images/000000245576.jpg"
Image(url=URL)

In [ ]:
# Ask the model what's in the picture.
conversation_history=[{
            "type": "human",
            "text": "What's in the photo?",
            "image_url": URL,
}]
response = reka.vlm_chat(conversation_history)
print(response['text'])

In [ ]:
# Continue the conversation.
conversation_history += [{"type": "model", "text": response['text']},
                        {"type": "human", "text": "What color is the cat?"}]
response = reka.vlm_chat(conversation_history)
print(response['text'])